# Kernel Methods Data Challenge

Authors:

Breno BALDAS SKUK & Samuel ASSERPE

# 1. Downloading Data

In [ ]:
import pandas as pd
import numpy as np
from scipy import linalg

In [ ]:
# download data from google drive if in google colab
google_colab = False
data_dir = ""
if google_colab:
  from google.colab import files
  from google_drive_downloader import GoogleDriveDownloader as gdd

  gdd.download_file_from_google_drive(file_id='1um-WlzyVqtTrJ0AQ2LX28Ailj-yLgme8',
                                    dest_path='./data.zip',
                                    unzip=True)

  # delete the .zip and readme
  !rm './data.zip'

else:
  data_dir = "data/"

Unzipping...Done.


In [ ]:
# 1st dataset
df_train_labels_0 = pd.read_csv(data_dir+"Ytr0.csv")
train_labels_0 = np.ravel(df_train_labels_0['Bound'].to_numpy())

# 2nd dataset 
df_train_labels_1 = pd.read_csv(data_dir+"Ytr1.csv")
train_labels_1 = np.ravel(df_train_labels_1['Bound'].to_numpy())

# 3rd dataset
df_train_labels_2 = pd.read_csv(data_dir+"Ytr2.csv")
train_labels_2 = np.ravel(df_train_labels_2['Bound'].to_numpy())


# Load precomputed matrices

cols = [str(i) for i in range(100)] # create some col names

# 1st dataset
Xtr0_mat = pd.read_csv(data_dir+"Xtr0_mat100.csv", sep="\s+|;|:", names=cols, header=None, engine="python").to_numpy()
Xte0_mat = pd.read_csv(data_dir+"Xte0_mat100.csv", sep="\s+|;|:", names=cols, header=None, engine="python").to_numpy()

# 2nd dataset
Xtr1_mat = pd.read_csv(data_dir+"Xtr1_mat100.csv", sep="\s+|;|:", names=cols, header=None, engine="python").to_numpy()
Xte1_mat = pd.read_csv(data_dir+"Xte1_mat100.csv", sep="\s+|;|:", names=cols, header=None, engine="python").to_numpy()

# 3rd dataset
Xtr2_mat = pd.read_csv(data_dir+"Xtr2_mat100.csv", sep="\s+|;|:", names=cols, header=None, engine="python").to_numpy()
Xte2_mat = pd.read_csv(data_dir+"Xte2_mat100.csv", sep="\s+|;|:", names=cols, header=None, engine="python").to_numpy()

# reading the raw data with pandas

# 1st dataset
Xtr0 = pd.read_csv(data_dir+"Xtr0.csv")
Xtr0=Xtr0['seq'].tolist()

Xte0 = pd.read_csv(data_dir+"Xte0.csv")
Xte0=Xte0['seq'].tolist()

# 2nd dataset
Xtr1 = pd.read_csv(data_dir+"Xtr1.csv")
Xtr1=Xtr1['seq'].tolist()

Xte1 = pd.read_csv(data_dir+"Xte1.csv")
Xte1=Xte1['seq'].tolist()

# 3rd dataset
Xtr2 = pd.read_csv(data_dir+"Xtr2.csv")
Xtr2=Xtr2['seq'].tolist()

Xte2 = pd.read_csv(data_dir+"Xte2.csv")
Xte2=Xte2['seq'].tolist()

# 2. Helper Functions

In [ ]:
def export_predict(pred_1,pred_2,pred_3,file_name):
    preds = pd.DataFrame({'Id':np.array(range(3000)),'Bound':np.concatenate([pred_1,pred_2,pred_3])})
    preds.to_csv(file_name,index=False)


def ensure_2D(array):

    # If input is scalar raise error
    if array.ndim == 0:
        raise ValueError(
            "Expected 2D array, got scalar array instead:\narray={}.\n"
            "Reshape your data either using array.reshape(-1, 1) if "
            "your data has a single feature or array.reshape(1, -1) "
            "if it contains a single sample.".format(array))
    # If input is 1D raise error
    if array.ndim == 1:
        raise ValueError(
            "Expected 2D array, got 1D array instead:\narray={}.\n"
            "Reshape your data either using array.reshape(-1, 1) if "
            "your data has a single feature or array.reshape(1, -1) "
            "if it contains a single sample.".format(array))
        
def check_y(X, y):

    y = y.reshape(-1)
    if len(y) != len(X):
      raise ValueError(
            "Expected arrayof shape ({},) or ({},1). Got instead:\narray={}."
            .format(len(X), len(X), array))
    

In [ ]:

def train_test_split(X, y, train_frac=0.75):
    
    shuffle = np.random.permutation(len(X))
    last_train = int(train_frac*len(X))
    
    X_rnd = X[shuffle]
    y_rnd = y[shuffle]

    return X_rnd[:last_train], y_rnd[:last_train], X_rnd[last_train:], y_rnd[last_train:]


def train_test_split_on_gram(K, y, train_frac=0.75):

    shuffle = np.random.permutation(len(K))
    last_train = int(train_frac*len(K))
    idxs_train = shuffle[:last_train]
    idxs_test = shuffle[last_train:]

    K_train = K[idxs_train].T[idxs_train].T
    K_test_train = K[idxs_test].T[idxs_train].T

    return K_train, y[idxs_train], K_test_train, y[idxs_test]



def accuracy(y_true, y_pred):
    assert len(y_true) == len(y_pred)
    return np.sum(y_true == y_pred) / len(y_true)



def remap_zero_minus_one(y):
    new_y = y.copy()
    new_y[y==0] = -1
    return new_y

def remap_minus_one_zero(y):
    new_y = y.copy()
    new_y[y==-1] = 0
    return new_y


# 3. Ridge Regression

In [ ]:


#Ridge without kernel
def ridgeEstimator(X,y,lambd):
    """
    Params:
    ######
    X      is a (n_samples, n_features) array : the training inputs
    y      is a (n_samples,) array : the training labels
    lambd  is a postive scalar (regularization term)

    Returns:
    ######
    theta : a (n_features,) array. The parameters of the linear model estimation
    """
    
    # check size of X
    ensure_2D(X)
    # check size of y
    check_y(X,y)

    # the labels must be {-1,+1}
    y = remap_zero_minus_one(y)
    
    (n,d) = X.shape
    # decide to invert a (n,n) or a (d,d) matrix
    if d <= n: # standard ridge regression
    
        # compute the ridge estimator
        cov = X.T @ X
        reg = lambd * n * np.eye(d)

        theta = linalg.solve( cov+reg , X.T @ y ) # résout le système linéaire
    
    else: # kernel ridge regression with a linear kernel

        gram = X @ X.T
        reg = lambd * n * np.eye(n)

        theta = X.T @ linalg.solve( gram+reg , y ) # résout le système linéaire

    return theta


def predictLinear(X, theta):

    # check size of X
    ensure_2D(X)
    # linear prediction
    preds = np.sign(X @ theta).astype(int)
    # remap {-1,1} to {0,1}
    preds = remap_minus_one_zero(preds)

    return preds


## Test classic ridge regression and hyperparameter research

In [ ]:
### Test classic ridge regression and hyperparameter research

n_iter = 100

# center the design matrix
(n,d) = Xtr0_mat.shape
means = np.ones(n) @ Xtr0_mat / n
Xtr0_mat_c = Xtr0_mat - means

for lambd in [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2]: # tout est un peu équivalent

  acc_train = 0
  acc_valid = 0
  
  for it in range(n_iter):
    X_train, y_train, X_valid, y_valid = train_test_split(Xtr0_mat_c, train_labels_0, train_frac=0.75)

    #ridge predictions:
    theta = ridgeEstimator(X_train, y_train, lambd=lambd)
    y_train_pred = predictLinear(X_train,theta)
    y_valid_pred = predictLinear(X_valid,theta)

    # evaluation
    acc_train += accuracy(y_train, y_train_pred)
    acc_valid += accuracy(y_valid, y_valid_pred)
  
  acc_train /= n_iter
  acc_valid /= n_iter

  print("\nWith lambda = {}\n".format(lambd))
  print("Accuracy on the train set : {} %".format((100 * acc_train)))
  print("Accuracy on the validation set : {} %".format((100 * acc_valid)))



With lambda = 1e-08

Accuracy on the train set : 66.23266666666669 %
Accuracy on the validation set : 58.777999999999984 %

With lambda = 1e-07

Accuracy on the train set : 66.22933333333333 %
Accuracy on the validation set : 58.46000000000002 %

With lambda = 1e-06

Accuracy on the train set : 66.11533333333334 %
Accuracy on the validation set : 58.92400000000002 %

With lambda = 1e-05

Accuracy on the train set : 65.99000000000001 %
Accuracy on the validation set : 58.67600000000001 %

With lambda = 0.0001

Accuracy on the train set : 64.64533333333334 %
Accuracy on the validation set : 59.233999999999995 %

With lambda = 0.001

Accuracy on the train set : 62.84866666666669 %
Accuracy on the validation set : 58.42 %

With lambda = 0.01

Accuracy on the train set : 62.165333333333315 %
Accuracy on the validation set : 58.240000000000016 %


## Sklearn Ridge regression results for comparaison

In [ ]:
### Compare with sklearn's ridge regression

from sklearn.linear_model import Ridge

n_iter = 100

for lambd in [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2]: # tout est un peu équivalent

  acc_train = 0
  acc_valid = 0
  
  for it in range(n_iter):
    X_train, y_train, X_valid, y_valid = train_test_split(Xtr0_mat_c, train_labels_0, train_frac=0.75)

    #ridge predictions:
    clf = Ridge(alpha=lambd * len(X_train), fit_intercept=False).fit(X_train, remap_zero_minus_one(y_train))
    y_train_pred = remap_minus_one_zero(np.sign(clf.predict(X_train)).astype(int))
    y_valid_pred = remap_minus_one_zero(np.sign(clf.predict(X_valid)).astype(int))

    # evaluation
    acc_train += accuracy(y_train, y_train_pred)
    acc_valid += accuracy(y_valid, y_valid_pred)
  
  acc_train /= n_iter
  acc_valid /= n_iter

  print("\nWith lambda = {}\n".format(lambd))
  print("Accuracy on the train set : {} %".format((100 * acc_train)))
  print("Accuracy on the validation set : {} %".format((100 * acc_valid)))



With lambda = 1e-08

Accuracy on the train set : 66.17133333333335 %
Accuracy on the validation set : 58.90600000000003 %

With lambda = 1e-07

Accuracy on the train set : 66.0153333333333 %
Accuracy on the validation set : 58.95000000000002 %

With lambda = 1e-06

Accuracy on the train set : 66.26800000000001 %
Accuracy on the validation set : 58.79199999999999 %

With lambda = 1e-05

Accuracy on the train set : 66.12133333333333 %
Accuracy on the validation set : 58.928000000000026 %

With lambda = 0.0001

Accuracy on the train set : 64.71866666666668 %
Accuracy on the validation set : 59.075999999999986 %

With lambda = 0.001

Accuracy on the train set : 62.751333333333335 %
Accuracy on the validation set : 59.01399999999999 %

With lambda = 0.01

Accuracy on the train set : 62.042000000000016 %
Accuracy on the validation set : 58.638 %


# 4. KERNEL Ridge Regression

In [ ]:
### Kernels on the matrix representation


### GAUSSIAN KERNEL
def gaussian_kernel(x, y, gamma=1):
    return np.exp(-gamma * np.sum((x - y)**2))

### LAPLACIAN KERNEL
def laplacian_kernel(x, y, gamma=1):
    return np.exp(-gamma * np.sum(abs(x - y)))

### POLYNOMIAL KERNEL
def ploynomial_kernel(x, y, d=3, gamma=1, c=1):
    return (gamma * x@y + c)**d

    
############################### K MATRICES ####################################


def compute_gram_from_kernel(X1,X2, kernel_func, params):
    n1 = len(X1)
    n2 = len(X2)
    K = np.zeros((n1,n2))
    for i in range(n1):
        for j in range(n2):
            K[i][j] = kernel_func(X1[i],X2[j], params)
    return K


########################################## KRR #################################


In [ ]:

def kernelRidgeEstimator(K,y,lambd):
    """
    Params:
    ######
    K : (n_samples, n_samples) array. Kernel matrix of the input training data
    y : (n_samples,) array. Training labels
    lambd : positive scalar. Regularization parameter
    
    Returns:
    ######
    alpha : (n_samples,) array. The weights to give to each training data point.
    """
    n = len(K)

    # the labels must be {-1,+1}
    y = remap_zero_minus_one(y)

    reg = lambd * n * np.eye(n)
    
    alpha = linalg.solve( K + reg , y ) # solve the linear system
    
    return alpha

def predict_from_gram(K_test_train, alpha):

    preds = np.sign(K_test_train @ alpha).astype(int)
    
    # remap {-1,1} to {0,1}
    preds = remap_minus_one_zero(preds)
  
    return preds


## Test KRR with gaussian kernel (and hyperparameters research)

In [ ]:
# Test KRR with gaussian kernel (and hyperparameters research)

need_to_compute_Grams = False

if need_to_compute_Grams: # take a long time with actual implementation of the Gaussian gram matrix
  Grams = []
  gammas = [1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]

  for gamma in gammas:

    Grams.append( compute_gram_matrix(Xtr0_mat, Xtr0_mat, gaussian_kernel, gamma) )


n_iter = 100

for i in range(len(gammas)):

  # global gram matrix
  K = Grams[i]

  # center the Gram matrix
  n = len(K)
  I = np.eye(n)
  U = np.ones((n,n)) / n
  K = (I-U) @ K @ (I-U)

  print("####################\n\nWith a gaussian kernel of parameter gamma = {}\n\n####################".format(gammas[i]))

  for lambd in [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2]: 

    acc_train = 0
    acc_valid = 0
    
    for it in range(n_iter):
      K_train, y_train, K_valid_train, y_valid = train_test_split_on_gram(K, train_labels_0, train_frac=0.75)

      #KRR predictions:
      alpha = kernelRidgeEstimator(K_train, y_train, lambd=lambd)
      y_train_pred = predict_from_gram(K_train, alpha)
      y_valid_pred = predict_from_gram(K_valid_train, alpha)

      # evaluation
      acc_train += accuracy(y_train, y_train_pred)
      acc_valid += accuracy(y_valid, y_valid_pred)
    
    acc_train /= n_iter
    acc_valid /= n_iter

    print("\nWith lambda = {}\n".format(lambd))
    print("Accuracy on the train set : {} %".format((100 * acc_train)))
    print("Accuracy on the validation set : {} %".format((100 * acc_valid)))



####################

With a gaussian kernel of parameter gamma = 0.001

####################

With lambda = 1e-08

Accuracy on the train set : 66.10333333333334 %
Accuracy on the validation set : 58.79999999999998 %

With lambda = 1e-07

Accuracy on the train set : 65.28199999999998 %
Accuracy on the validation set : 59.297999999999995 %

With lambda = 1e-06

Accuracy on the train set : 63.266 %
Accuracy on the validation set : 59.07800000000003 %

With lambda = 1e-05

Accuracy on the train set : 62.19733333333334 %
Accuracy on the validation set : 58.782000000000025 %

With lambda = 0.0001

Accuracy on the train set : 62.15133333333333 %
Accuracy on the validation set : 58.184000000000005 %

With lambda = 0.001

Accuracy on the train set : 61.84933333333335 %
Accuracy on the validation set : 58.147999999999996 %

With lambda = 0.01

Accuracy on the train set : 61.87133333333335 %
Accuracy on the validation set : 58.156000000000006 %
####################

With a gaussian kernel of par

## Sklearn Kernel Ridge regression results for comparaison

In [ ]:
### Compare with sklearn's Kernel ridge regression

from sklearn.kernel_ridge import KernelRidge

n_iter = 100

for i in range(len(gammas)):

  # global gram matrix
  K = Grams[i]

  # center the Gram matrix
  n = len(K)
  I = np.eye(n)
  U = np.ones((n,n)) / n
  K = (I-U) @ K @ (I-U)

  print("####################\n\nWith a gaussian kernel of parameter gamma = {}\n\n####################".format(gammas[i]))

  for lambd in [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2]: 

    acc_train = 0
    acc_valid = 0
    
    for it in range(n_iter):
      K_train, y_train, K_valid_train, y_valid = train_test_split_on_gram(K, train_labels_0, train_frac=0.75)

      #KRR predictions:
      clf = KernelRidge(alpha=lambd * len(K_train), kernel="precomputed").fit(K_train, remap_zero_minus_one(y_train))
      y_train_pred = remap_minus_one_zero(np.sign(clf.predict(K_train)).astype(int))
      y_valid_pred = remap_minus_one_zero(np.sign(clf.predict(K_valid_train)).astype(int))

      # evaluation
      acc_train += accuracy(y_train, y_train_pred)
      acc_valid += accuracy(y_valid, y_valid_pred)
    
    acc_train /= n_iter
    acc_valid /= n_iter

    print("\nWith lambda = {}\n".format(lambd))
    print("Accuracy on the train set : {} %".format((100 * acc_train)))
    print("Accuracy on the validation set : {} %".format((100 * acc_valid)))



####################

With a gaussian kernel of parameter gamma = 0.001

####################

With lambda = 1e-08

Accuracy on the train set : 65.98333333333335 %
Accuracy on the validation set : 59.094 %

With lambda = 1e-07

Accuracy on the train set : 65.30799999999999 %
Accuracy on the validation set : 59.30399999999998 %

With lambda = 1e-06

Accuracy on the train set : 63.16333333333334 %
Accuracy on the validation set : 59.02 %

With lambda = 1e-05

Accuracy on the train set : 62.13199999999999 %
Accuracy on the validation set : 58.628000000000014 %

With lambda = 0.0001

Accuracy on the train set : 62.03733333333332 %
Accuracy on the validation set : 58.05999999999998 %

With lambda = 0.001

Accuracy on the train set : 61.783999999999985 %
Accuracy on the validation set : 58.053999999999995 %

With lambda = 0.01

Accuracy on the train set : 61.901333333333305 %
Accuracy on the validation set : 58.71200000000001 %
####################

With a gaussian kernel of parameter gamma 

# 6. Spectrum Kernel for raw sequences

In [ ]:

######################################################
### SPECTRUM KERNEL
######################################################

def spectrum_kernel(x1,x2,k=6, normalize=False):
    
    # initialize the dictionaries of subsequences
    phi1=dict()
    phi2=dict()
    
    for i in range(len(x1)-k+1):
        
        # extract a subsequence of x1
        subsequence = x1[i:k+i]
        
        # add the subsequence found to the dictionary 1
        if subsequence not in phi1:
            phi1[subsequence] = 1
        else:
            phi1[subsequence] += 1
        
    for i in range(len(x2)-k+1):

        # extract a subsequence of x2
        subsequence = x2[i:k+i]
        
        # add the subsequence found to the dictionary 2
        if subsequence not in phi2:
            phi2[subsequence]=1
        else:
            phi2[subsequence]+=1
    
    # compute the scalar product between the two representations obtained
    scalar_prod = 0
    for subsequence in phi1.keys() & phi2.keys():
        scalar_prod += phi1[subsequence] * phi2[subsequence]
    
    if normalize:
      norm_phi1 = ( sum(e**2 for e in phi1.values()) )**0.5
      norm_phi2 = ( sum(e**2 for e in phi2.values()) )**0.5

      scalar_prod /= (norm_phi1 * norm_phi2)
    
    return scalar_product



#########################################################
### FASTER IMPLEMENTATION TO COMPUTE THE GRAM MATRIX
#########################################################

def compute_spectrum_kernel_Gram(X,k=6, normalize=False):
  """
  implement the function above but with ina  slightly more efficient version

  In X, each row represents a sequence ('ATCGCTTGA...)
  """

    # initialize the dictionaries of subsequences
  phis = []
  for sequence_id in range(len(X)):

    phi = dict()
    
    for i in range(len(X[sequence_id])-k+1):
        
      # extract a subsequence
      subsequence = X[sequence_id][i:k+i]
      
      # add the subsequence found to its dictionary 
      if subsequence not in phi:
        phi[subsequence] = 1
      else:
        phi[subsequence] += 1

    phis.append(phi)

  norms_phis = np.array([(sum(e**2 for e in phis[i].values()))**0.5 for i in range(len(phis))]).reshape(1, -1)
  
  Gram = np.zeros((len(X),len(X)))
  for i in range(len(X)):
    for j in range(i+1,len(X)):

      # compute the scalar products between the representations obtained
      scalar_prod = 0
      for subsequence in phis[i].keys() & phis[j].keys():
        scalar_prod += phis[i][subsequence] * phis[j][subsequence]
      
      Gram[i,j] = scalar_prod
      Gram[j,i] = scalar_prod
  
  if normalize:
    Gram = Gram / np.repeat(norms_phis, len(X), axis=0)
    Gram = Gram / np.repeat(norms_phis, len(X), axis=0).T
    Gram = Gram + np.eye(len(X))
  else:
    Gram = Gram + np.diag(norms_phis.reshape(-1))
  
  return Gram



#######################################################################
### COMPUTE THE SUM OF GRAM MATRICES USING SUMS OF SPECTRUM KERNELS
#######################################################################

def compute_sum_spectrum_kernels_Gram(X, list_k, normalize=False):
  """
  Allows to use the spectrum kernel for multiple subsequences-lenghts and sum the kernels
  """

  Gram = np.zeros((len(X),len(X)))
  for k in list_k:
    Gram = Gram + compute_spectrum_kernel_Gram(X,k,normalize)

  return Gram / len(list_k)


## Test KRR with spectrum kernel

In [ ]:
# Test KRR with gaussian kernel (and hyperparameters research)

# global gram matrix

K = compute_sum_spectrum_kernels_Gram(Xtr0, list_k=[6,7,8], normalize=True)

n_iter = 100

# center the Gram matrix
n = len(K)
I = np.eye(n)
U = np.ones((n,n)) / n
K = (I-U) @ K @ (I-U)

print("####################\n\nWith a sum of spectrum kernels of parameters k = {}\n\n####################".format(list_k))

for lambd in [1e-4, 0.0003, 1e-3, 0.003, 1e-2, 0.03, 1e-1]: 

  acc_train = 0
  acc_valid = 0
  
  for it in range(n_iter):
    K_train, y_train, K_valid_train, y_valid = train_test_split_on_gram(K, train_labels_0, train_frac=0.75)

    #KRR predictions:
    alpha = kernelRidgeEstimator(K_train, y_train, lambd=lambd)
    y_train_pred = predict_from_gram(K_train, alpha)
    y_valid_pred = predict_from_gram(K_valid_train, alpha)

    # evaluation
    acc_train += accuracy(y_train, y_train_pred)
    acc_valid += accuracy(y_valid, y_valid_pred)
  
  acc_train /= n_iter
  acc_valid /= n_iter

  print("\nWith lambda = {}\n".format(lambd))
  print("Accuracy on the train set : {} %".format((100 * acc_train)))
  print("Accuracy on the validation set : {} %".format((100 * acc_valid)))



####################

With a sum of spectrum kernels of parameters k = [5, 6, 7, 7, 8, 8]

####################

With lambda = 0.0001

Accuracy on the train set : 100.0 %
Accuracy on the validation set : 63.47600000000002 %

With lambda = 0.0003

Accuracy on the train set : 99.79000000000006 %
Accuracy on the validation set : 64.052 %

With lambda = 0.001

Accuracy on the train set : 98.39066666666668 %
Accuracy on the validation set : 64.42999999999999 %

With lambda = 0.003

Accuracy on the train set : 92.43400000000004 %
Accuracy on the validation set : 64.514 %

With lambda = 0.01

Accuracy on the train set : 82.17066666666669 %
Accuracy on the validation set : 62.694 %

With lambda = 0.03

Accuracy on the train set : 75.19600000000004 %
Accuracy on the validation set : 62.18799999999997 %

With lambda = 0.1

Accuracy on the train set : 72.09333333333332 %
Accuracy on the validation set : 61.682000000000016 %


#### The spectrum kernel on the raw DNA sequences is better than the Gaussian on the precomputed matrix.

#### Plus, when we sum some spectrum kernels, we gat again better results.

#### Doing Boostrap does not seem to enhance the classification, since we use less training samples each times, the neighbours are more far from the test samples.

In [ ]:
# predictions for dataset 0

k_0 = [6,7,8]
lambd_0 = 0.001

X_te_plus_tr_0 = Xte0 + Xtr0

K_te_plus_tr_0 = compute_sum_spectrum_kernels_Gram(X_te_plus_tr_0, k_0, normalize=True)


# center the Gram matrix
n = len(K_te_plus_tr_0)
I = np.eye(n)
U = np.ones((n,n)) / n
K_te_plus_tr_0 = (I-U) @ K_te_plus_tr_0 @ (I-U)

K_tr_0 = K_te_plus_tr_0[1000:,1000:]
K_te_tr_0 = K_te_plus_tr_0[:1000,1000:]


#KRR predictions:
alpha_0 = kernelRidgeEstimator(K_tr_0, train_labels_0, lambd=lambd_0)

y_pred_0 = predict_from_gram(K_te_tr_0, alpha_0)



In [ ]:
# predictions for dataset 1

k_1 = [5,7,8]
lambd_1 = 0.001

X_te_plus_tr_1 = Xte1 + Xtr1

K_te_plus_tr_1 = compute_sum_spectrum_kernels_Gram(X_te_plus_tr_1, k_1, normalize=True)


# center the Gram matrix
n = len(K_te_plus_tr_1)
I = np.eye(n)
U = np.ones((n,n)) / n
K_te_plus_tr_1 = (I-U) @ K_te_plus_tr_1 @ (I-U)

K_tr_1 = K_te_plus_tr_1[1000:,1000:]
K_te_tr_1 = K_te_plus_tr_1[:1000,1000:]


#KRR predictions:
alpha_1 = kernelRidgeEstimator(K_tr_1, train_labels_1, lambd=lambd_1)

y_pred_1 = predict_from_gram(K_te_tr_1, alpha_1)



In [ ]:
# predictions for dataset 2

k_2 = [6,8]
lambd_2 = 0.0003

X_te_plus_tr_2 = Xte2 + Xtr2

K_te_plus_tr_2 = compute_sum_spectrum_kernels_Gram(X_te_plus_tr_2, k_2, normalize=True)


# center the Gram matrix
n = len(K_te_plus_tr_2)
I = np.eye(n)
U = np.ones((n,n)) / n
K_te_plus_tr_2 = (I-U) @ K_te_plus_tr_2 @ (I-U)

K_tr_2 = K_te_plus_tr_2[1000:,1000:]
K_te_tr_2 = K_te_plus_tr_2[:1000,1000:]


#KRR predictions:
alpha_2 = kernelRidgeEstimator(K_tr_2, train_labels_2, lambd=lambd_2)

y_pred_2 = predict_from_gram(K_te_tr_2, alpha_2)



In [ ]:

export_predict(y_pred_0,y_pred_1,y_pred_2,'preds_sum_kernels.csv')


### Ensemble methods

In [ ]:
# predictions for dataset 0

k_0 = [6,7,8]
lambd_0 = 0.001


X_te_plus_tr_0 = Xte0 + Xtr0

K_te_plus_tr_0 = compute_sum_spectrum_kernels_Gram(X_te_plus_tr_0, k_0, normalize=True)

y_pred_0 = []

for n_model in range(15):

  shuffle = np.random.permutation(2000)
  last_train = int(0.75*2000)
  idxs_train = shuffle[:last_train]
  idxs = np.concatenate([np.arange(1000), 1000+idxs_train])

  K_te_plus_part_tr_0 = K_te_plus_tr_0[idxs].T[idxs].T
  part_train_labels_0 = train_labels_0[idxs_train]

  # center the Gram matrix
  n = len(K_te_plus_part_tr_0)
  I = np.eye(n)
  U = np.ones((n,n)) / n
  K_te_plus_part_tr_0 = (I-U) @ K_te_plus_part_tr_0 @ (I-U)

  K_part_tr_0 = K_te_plus_part_tr_0[1000:,1000:]
  K_te_part_tr_0 = K_te_plus_part_tr_0[:1000,1000:]


  #KRR predictions:
  alpha_0 = kernelRidgeEstimator(K_part_tr_0, part_train_labels_0, lambd=lambd_0)

  y_pred_0.append(predict_from_gram(K_te_part_tr_0, alpha_0))

y_pred_0 = np.array(y_pred_0)
y_pred_merged_0 = np.round(y_pred_0.mean(axis=0)).astype(int)


In [ ]:
# predictions for dataset 1

k_1 = [5,7,8]
lambd_1 = 0.001


X_te_plus_tr_1 = Xte1 + Xtr1

K_te_plus_tr_1 = compute_sum_spectrum_kernels_Gram(X_te_plus_tr_1, k_1, normalize=True)

y_pred_1 = []

for n_model in range(15):

  shuffle = np.random.permutation(2000)
  last_train = int(0.75*2000)
  idxs_train = shuffle[:last_train]
  idxs = np.concatenate([np.arange(1000), 1000+idxs_train])

  K_te_plus_part_tr_1 = K_te_plus_tr_1[idxs].T[idxs].T
  part_train_labels_1 = train_labels_1[idxs_train]

  # center the Gram matrix
  n = len(K_te_plus_part_tr_1)
  I = np.eye(n)
  U = np.ones((n,n)) / n
  K_te_plus_part_tr_1 = (I-U) @ K_te_plus_part_tr_1 @ (I-U)

  K_part_tr_1 = K_te_plus_part_tr_1[1000:,1000:]
  K_te_part_tr_1 = K_te_plus_part_tr_1[:1000,1000:]


  #KRR predictions:
  alpha_1 = kernelRidgeEstimator(K_part_tr_1, part_train_labels_1, lambd=lambd_1)

  y_pred_1.append(predict_from_gram(K_te_part_tr_1, alpha_1))

y_pred_1 = np.array(y_pred_1)
y_pred_merged_1 = np.round(y_pred_1.mean(axis=0)).astype(int)


In [ ]:
# predictions for dataset 2

k_2 = [6,8]
lambd_2 = 0.0003


X_te_plus_tr_2 = Xte2 + Xtr2

K_te_plus_tr_2 = compute_sum_spectrum_kernels_Gram(X_te_plus_tr_2, k_2, normalize=True)

y_pred_2 = []

for n_model in range(15):

  shuffle = np.random.permutation(2000)
  last_train = int(0.75*2000)
  idxs_train = shuffle[:last_train]
  idxs = np.concatenate([np.arange(1000), 1000+idxs_train])

  K_te_plus_part_tr_2 = K_te_plus_tr_2[idxs].T[idxs].T
  part_train_labels_2 = train_labels_2[idxs_train]

  # center the Gram matrix
  n = len(K_te_plus_part_tr_2)
  I = np.eye(n)
  U = np.ones((n,n)) / n
  K_te_plus_part_tr_2 = (I-U) @ K_te_plus_part_tr_2 @ (I-U)

  K_part_tr_2 = K_te_plus_part_tr_2[1000:,1000:]
  K_te_part_tr_2 = K_te_plus_part_tr_2[:1000,1000:]


  #KRR predictions:
  alpha_2 = kernelRidgeEstimator(K_part_tr_2, part_train_labels_2, lambd=lambd_2)

  y_pred_2.append(predict_from_gram(K_te_part_tr_2, alpha_2))

y_pred_2 = np.array(y_pred_2)
y_pred_merged_2 = np.round(y_pred_2.mean(axis=0)).astype(int)


In [ ]:

export_predict(y_pred_merged_0,y_pred_merged_1,y_pred_merged_2,'preds_sum_kernels_ensemble.csv')
